In [1]:
#импорт библиотек, загрузка модели, весов, разметки клаассов
import torch
import numpy as np
import pandas as pd
device = torch.device("cpu")
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AdamW, get_scheduler
model_name = "sberbank-ai/sbert_large_nlu_ru"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=660).to(device)
from sklearn import preprocessing
Label_encoder = preprocessing.LabelEncoder()
Label_encoder.classes_ = np.load('./cl_classes2306.npy', allow_pickle=True)
model.load_state_dict(torch.load("../../best_model2306/pytorch_model.bin", map_location=device))

C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/sbert_large_nlu_ru and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [2]:
#импорт описания классов (выдрал с альта софта)
dscr = pd.read_csv("../data/desc4d.csv", sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})

In [3]:
#ищем классы нейросети, к которым нет описания.
withlabels = list(dscr['id'])
justids = list(Label_encoder.classes_)
for each in justids:
    if each not in withlabels:
        print(each)

0391
0481
0830
1012
1018
1062
1068
5022
5032
5035
5052
5058
5075
6908


In [4]:
#функция предсказания одного класса (аргмакс). По умолчанию с кодом класса дает описание класса
def predict_class(text, desc=True):
    model.to(torch.device('cpu'))
    inputs = tokenizer(text, truncation = True, max_length=100, padding='max_length', return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
        predicted_class_id = logits.argmax().item()
        if desc==True:
            result = dict()
            id = Label_encoder.inverse_transform([predicted_class_id])[0]
            result[id] = dscr[dscr['id']==id].iloc[0]['label']
            return result
        else:
            return Label_encoder.inverse_transform([predicted_class_id])[0]

In [5]:
#Пример с описанием
predict_class('КОНДИТЕРСКИЕ ИЗДЕЛИЯ В ВИДЕ РЕЗИНКИ И ЖЕЛЕ, НЕ СОДЕРЖАЩИЕ КАКАО')

{'1704': 'Кондитерские изделия из сахара (включая белый шоколад), не содержащие какао'}

In [6]:
#Пример без описания
predict_class('КОНДИТЕРСКИЕ ИЗДЕЛИЯ В ВИДЕ РЕЗИНКИ И ЖЕЛЕ, НЕ СОДЕРЖАЩИЕ КАКАО', desc=False)

'1704'

In [7]:
#функция предикта классов с вероятностью (софтмакс), по умолчанию 5 классов
def predict_prob(text, qtty=5):
    model.to(torch.device('cpu'))
    inputs = tokenizer(text, truncation = True, max_length=100, padding='max_length', return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    result = dict()
    p = torch.nn.functional.softmax(logits, dim=1)
    for i in range(qtty):
        a = p.argmax().item()
        result[Label_encoder.inverse_transform([a])[0]] = p[0][a].item()
        p[0][a] = 0
    return result

In [8]:
#Пример с 3 классами
print(predict_prob('трубы сантехнические пластиковые ', 3))

{'3917': 0.7268840670585632, '7304': 0.01337417121976614, '4009': 0.010102425701916218}


In [9]:
#Пример с 5 классами
print(predict_prob('трубы металлические ', 5))

{'7304': 0.9046436548233032, '3917': 0.010384665802121162, '7306': 0.00989900715649128, '8307': 0.003736451966688037, '7307': 0.0020984946750104427}


In [10]:
#функция предикта классов и описания с вероятностью, по умолчанию 5 шт. Использует функцию predict_prob.
#возвращает список списков
def predict_prob_with_descr(text, qtty=5):
    probs = predict_prob(text, qtty=qtty)
    #result = np.array()
    result = list()
    for each in probs:
        result.append([each, dscr[dscr['id']==each].iloc[0]['label'], probs[each]])
    return result

In [11]:
#Пример с 5 классами.
predict_prob_with_descr('покрышки')

[['4011',
  'Шины и покрышки пневматические резиновые новые',
  0.7416127324104309],
 ['4016',
  'Изделия из вулканизованной резины, кроме твердой резины, прочие',
  0.02087979018688202],
 ['8708',
  'Части и принадлежности моторных транспортных средств товарных позиций 8701 - 8705',
  0.011835361830890179],
 ['9401',
  'Мебель для сидения (кроме указанной в товарной позиции 9402), трансформируемая или не трансформируемая в кровати, и ее части',
  0.011823809705674648],
 ['7312',
  'Скрученная проволока, тросы, канаты, плетеные шнуры, стропы и аналогичные изделия, из черных металлов, без электрической изоляции',
  0.009819085709750652]]

In [12]:
#причесанный в пандас пример
pd.DataFrame(predict_prob_with_descr('орехи'), columns=['ID', 'LABEL', 'PROB'])

,ID,LABEL,PROB
0,1202,"Арахис, нежареный или не приготовленный каким-...",0.024651
1,0808,"Яблоки, груши и айва, свежие",0.014857
2,2001,"Овощи, фрукты, орехи и другие съедобные части ...",0.012068
3,8311,"Проволока, прутки, трубы, пластины, электроды ...",0.010070
4,8506,Первичные элементы и первичные батареи,0.009615


In [13]:
def consolelog(probs):
    print('prob-ty', 'Code', 'Description', sep=':    ')
    #print('____________________________________________________________________')
    for each in probs:
        if len(each[1]) < 40:
            des = each[1]
        else:
            des = each[1][:40] + '...'
        print(f"{each[2]:.{5}f}", each[0], des, sep=':    ')
        #print('____________________________________________________________________')

In [16]:
consolelog(predict_prob_with_descr("гандоны", 5))

prob-ty:    Code:    Description
0.06506:    9603:    Метлы, щетки (включая щетки, являющиеся ...
0.02958:    9619:    [с 01.01.22] Женские гигиенические прокл...
0.02412:    9503:    Трехколесные велосипеды, самокаты, педал...
0.01984:    9404:    Основы матрацные; принадлежности постель...
0.01886:    8714:    Части и принадлежности к транспортным ср...


In [15]:
#загрузим обучающий датасет
df = pd.read_csv("../data/mergedcleared0407.csv", sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})
#df = pd.read_csv("../data/firstset.csv", sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})
#функция для вывода обучающих данных по коду. При pandas=False возвращает numpy массив
def whats_data(id, pandas=True):
    if pandas == True:
        return df[df.id.str.slice(start=0, stop=len(str(id)))==str(id)]
    else:
        return df[df.id.str.slice(start=0, stop=len(str(id)))==str(id)].to_numpy()

In [43]:
#пример использования
whats_data('0803')

,id,label
41050,0803901000,бананы свежие для продовольственных целей урож...
41051,0803901000,бананы свежие для продовольственных целей фир...
41052,0803901000,бананы свежие зеленые сорт cavendish 19812 06к...
41053,0803901000,бананы свежие зелёные сорт cavendish класс 1 р...
41054,0803901000,бананы свежие 2 сорт фирма роялфрут с а tm отс...
41055,0803901000,бананы свежие урожая 2019г в карт коробках 192...
41056,0803901000,бананы свежие зеленые сорт cavendish кавендиш ...
41057,0803901000,бананы свежие зелёные сорт cavendish класс 1 р...
41058,0803901000,бананы свежие фирма неизвестен tm отсутствует
41059,0803901000,бананы свежие урожай 2019 года количество 9900...


In [46]:
similar = pd.read_csv("../data/Similar.csv", sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})
#similar['labels'] = Label_encoder.transform(similar['id'])
#Label_encoder.transform(['8474'])

In [44]:
predict_prob_with_descr('бананы свежие')

[['0808', 'Яблоки, груши и айва, свежие', 0.05644277110695839],
 ['2001',
  'Овощи, фрукты, орехи и другие съедобные части растений, приготовленные или консервированные с добавлением уксуса или уксусной кислоты',
  0.029795827344059944],
 ['2003',
  'Грибы и трюфели, приготовленные или консервированные без добавления уксуса или уксусной кислоты',
  0.01593076065182686],
 ['0804',
  'Финики, инжир, ананасы, авокадо, гуайява, манго и мангостан, или гарциния, свежие или сушеные',
  0.01508325431495905],
 ['0805', 'Цитрусовые плоды, свежие или сушеные', 0.014903338626027107]]

In [47]:
similar['id'] = similar.id.str.slice(start=0, stop=4)
similar


,id,label
0,4011,шины
1,4011,шина
2,4011,авто шины
3,4011,чернитель шин
4,4011,тапочки резиновые
...,...,...
59752,8474,грохот sandvik
59753,8474,вибрационный стол
59754,8474,вибрационные грохоты для горнодобывающей промы...
59755,8474,аппарат для деления руды


In [49]:
similar[similar['id']=='0803']

,id,label
11847,0803,банан
11848,0803,бананы
11849,0803,бананы свежие
11850,0803,греча зеленая
11851,0803,зеленая масса
11852,0803,фян
11853,0803,банан пыль
11854,0803,банан свежий
11855,0803,зеленая масса сенаж
11856,0803,лизун банан брелок


In [20]:
similar['labels'] = Label_encoder.transform(similar['id'])
Label_encoder.transform(['8474'])

array([512])

In [21]:
similar.columns = ['id', 'description', 'labels']
similar = similar.drop(['id'], axis=1, inplace=False)
similar

,description,labels
0,шины,171
1,шина,171
2,авто шины,171
3,чернитель шин,171
4,тапочки резиновые,171
...,...,...
59752,грохот sandvik,512
59753,вибрационный стол,512
59754,вибрационные грохоты для горнодобывающей промы...,512
59755,аппарат для деления руды,512


In [22]:
similar.to_csv('../../../test.csv', index=False)

In [23]:
from datasets import load_dataset, load_metric
dataset = load_dataset('csv', data_files={'test': '../../../test.csv'})

Extracting data files: 100%|████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 199.89it/s]


Dataset csv downloaded and prepared to C:/Users/yeril/.cache/huggingface/datasets/csv/default-f2645ff1032bf66d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 99.78it/s]


In [24]:
dataset = dataset.map(lambda e: tokenizer(e['description'], truncation = True, max_length=100, padding='max_length'), batched=True)

In [25]:
pytorch_style_columns = ['input_ids', 'token_type_ids', 'attention_mask', 'labels']  # 
#dataset = dataset.remove_columns(set(dataset['train'].features.keys()) - set(pytorch_style_columns))

In [26]:
set(dataset['test'].features.keys())


{'attention_mask', 'description', 'input_ids', 'labels', 'token_type_ids'}

In [27]:
dataset.set_format(type='torch', columns=pytorch_style_columns, device='cuda') 

In [28]:
from torch.utils.data import DataLoader
test_dataloader = DataLoader(dataset['test'], shuffle=False, batch_size=2)


In [29]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [30]:
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024

In [31]:
from tqdm import tqdm

In [32]:
from sklearn import preprocessing
from sklearn.metrics import classification_report
#Label_encoder = preprocessing.LabelEncoder()
#Label_encoder.classes_ = np.load('./cl_classes1307.npy', allow_pickle=True)

true = []
preds = []

model.eval()
for batch in tqdm(test_dataloader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    true += batch["labels"].detach().cpu().numpy().tolist()
    preds += predictions.detach().cpu().numpy().tolist()

print(classification_report(Label_encoder.inverse_transform(true), Label_encoder.inverse_transform(preds)))

100%|████████████████████████████████████████████████████████████████████████████| 29879/29879 [54:04<00:00,  9.21it/s]
C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Pyt

              precision    recall  f1-score   support

        0207       0.00      0.00      0.00        50
        0304       0.00      0.00      0.00         2
        0305       0.00      0.00      0.00        50
        0604       0.00      0.00      0.00        11
        0701       0.00      0.00      0.00         3
        0703       0.00      0.00      0.00        16
        0712       0.00      0.00      0.00        31
        0713       0.00      0.00      0.00         7
        0801       0.00      0.00      0.00        12
        0802       0.00      0.00      0.00        19
        0803       0.00      0.00      0.00        14
        0804       1.00      0.08      0.15        38
        0805       0.00      0.00      0.00        98
        0806       0.00      0.00      0.00        26
        0808       0.35      0.58      0.44       119
        0901       0.00      0.00      0.00        84
        0902       0.00      0.00      0.00       221
        0910       0.00    

C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
